In [ ]:
### This script aims to calculate third cumulants (equal to three-fold covariances) between time-of-flight regions in real data.
#   It is adapted with permission from James Somper's third cumulance codes for simulated data, and it is hoped that this
#   adaptation for beamtime data may be useful for future beamtime analyses. It is intended that this code is run *after* radial
#   covariance calculations are run and after the ToF spectrum has already been obtained analysed.

In [ ]:
# Step 1: Importing some useful modules

import pandas as pd
import numpy as np
import math
import os
import re
import time
from numpy import exp, loadtxt, pi, sqrt
import matplotlib.cm as cm
import gc
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numba import njit

In [ ]:
# Step 2: Reading in the data. Creates a list of dataframes. Takes in .npy files as input.

pd.set_option('display.max_rows', 4096) # Controls number of lines displayed (PImMS maximum = 4096 timebins)

directory=os.chdir(r"/home/merrickj/Documents/indene_filtered_data_for_covariance")
#directory=os.chdir(r"/home/merrickj/Documents/fluorene_filtered_data_for_covariance")

#file_list = ['038','039','040','041','042','043','044'] # run numbers for triphenylene, 2021
file_list = ['indene_full_data_centroided_corrected']
#file_list = ['fluorene_full_data_centroided_corrected']

df_list = []

for f in zip(file_list):
    print(f[0])
    print('Opening file: '+f[0])
    data = np.load(f"{f[0]}.npy", mmap_mode='r')
    #df = pd.DataFrame(data, columns = ['Energy','tId','x','y','ToF','size','spread','delay','m_z'])
    df = pd.DataFrame(data, columns = ['Energy','tId','x','y','ToF','size','spread','delay'])
    df_list.append(df)
    del data
    gc.collect()
    print(df)
    
# for just reading in 1% of the data for testing purposes
#df = df.sort_values(by='tId')
#df_list = np.array_split(df,100)
#print(np.shape(df_list))
#df = df_list[0]
#print(df)

In [ ]:
# Step 3: PLotting ToF spectrum for (possible) ease of future plotting.

# directory=os.chdir(r"/home/merrickj/Documents/indene_ion_figs_and_cov_plots")

fig, ax = plt.subplots(figsize=(15, 6))
plt.xlabel('Time-of-flight / a.u.', fontsize=12)
plt.tick_params(axis='x', labelsize=12)
plt.ylabel('Intensity / a.u.', fontsize=12)
plt.tick_params(axis='y', labelsize=12)
ax.set_xlim(1550,1800) # alter as necessary
ax.grid(which='major')
ax.grid(which='minor')
ax.minorticks_on()
tof_appended = []
for df,file in zip(df_list,file_list):
    plt.title('Run-concatenated ToF spectrum - indene', fontsize = 16)
    df = df[(df['ToF']>1550) & (df['ToF']<1900)]
    tof = df.groupby(['ToF']).size().reset_index(name='intensity')
    tof_times_list = tof['ToF'].to_list()
    #print(tof)
    tof_appended.append(tof)
    plt.plot(tof.ToF,tof.intensity,label=str(file))
plt.show()
# directory=os.chdir(r"/home/merrickj/Documents/indene_ion_figs_and_cov_plots")

In [ ]:
# Step 4: Adapting cumulant calculation function for real beamtime data. Note that three-fold covariance is given by:
#         cov(A,B,C) = <abc>, where i = I - <i> (mean-scaled values)

import numba
from numba import jit
from joblib import Parallel, delayed
from numpy import random
# Function for ease of plotting - will take in ToF list as an input. Will need to find midpoints of ToF tuples for plotting.

def GeneratePlottableMassList3D(CountingArray1):
    XArray = []
    YArray = []
    ZArray = []
    for i in range(len(CountingArray1)):
        for j in range(len(CountingArray1)):
            for k in range(len(CountingArray1)):
                XArray.append(CountingArray1[i])
                YArray.append(CountingArray1[j])
                ZArray.append(CountingArray1[k])
    return XArray, YArray, ZArray

# PlottingData is what is actually plotted

def GenerateThirdCumulantPlottingData(list_of_lists):
    
    ### Loop to be parallelised
    def run_i_loops(i):
        print(i)
        PlottingArray_i = []
        CountingArray1 = list_of_lists[i]
        #print(f"Calculating set of cumulances {i} of {len(list_of_lists)}")
        for j in range(len(list_of_lists)):
            CountingArray2 = list_of_lists[j]
            for k in range(len(list_of_lists)):
                CountingArray3 = list_of_lists[k]
                #print(f"Calculating third cumulant: {count}")
                PlottingArray_i.append((CalculateThirdCumulant(CountingArray1, CountingArray2, CountingArray3),i))
        return (PlottingArray_i)
    ###
    ### Create list for each index in list_of_lists in the format (output_run_i_loops[i],i)
    ### For the test data that returns an array of shape 10,100,2 
    PlottingArray = Parallel(n_jobs=20, verbose=100)(delayed(run_i_loops)(i) for i in np.arange(len(list_of_lists)))

    return PlottingArray
# Calculates mean of an array

def CalculateSingleExpectationValue(CountingArray):
    ExpectationValue = np.sum(CountingArray)/len(CountingArray)
    return ExpectationValue

# Takes array of number of ions of a given type per shot and subtracts the mean number of ions of a given type per shot from each element in the array to get mean-scaled values

def CreateMeanScaledArray(CountingArray1):
    Expect1 = CalculateSingleExpectationValue(CountingArray1)
    MeanScaledArray = CountingArray1 - Expect1
    return MeanScaledArray

# Function which actually calculates the third cumulant of three given ions. Takes three arrays as inputs.

def CalculateThirdCumulant(CountingArray1, CountingArray2, CountingArray3):
    if len(CountingArray1) != len(CountingArray2) or len(CountingArray1) != len(CountingArray2):
        print("Cannot calculate third cumulant value due to insufficient data points in one array")
    else: 
        MeanScaled1 = CreateMeanScaledArray(CountingArray1)
        MeanScaled2 = CreateMeanScaledArray(CountingArray2)
        MeanScaled3 = CreateMeanScaledArray(CountingArray3)
        ThirdCumulant = np.sum(np.multiply(MeanScaled1, np.multiply(MeanScaled2, MeanScaled3))) / len(MeanScaled1)
        return ThirdCumulant

# Plots the third cumulants on a three-dimensional plot

def DisplayThirdCumulants(ToF_list_for_plotting):
    PlottingXArray = GeneratePlottableMassList3D(ToF_list_for_plotting)[0]
    #print(PlottingXArray)
    PlottingYArray = GeneratePlottableMassList3D(ToF_list_for_plotting)[1]
    #print(PlottingYArray)
    PlottingZArray = GeneratePlottableMassList3D(ToF_list_for_plotting)[2]
    #print(PlottingZArray)
    MaxValue = np.max(PlottingArray)
    
    return PlottingXArray, PlottingYArray, PlottingZArray, MaxValue

In [ ]:
# Step 5: Running the code! This is adapted with the possibility of contingent covariance, but this was found to severely reduce
#         the signal-to-noise ratio. By setting number of bins to 2, covariance is calculated without contingency.

# N.B.: Contingent covariance NOT implemented in this code.

minToF = 1550
maxToF = 1800
points1 = np.arange(minToF-0.5, maxToF-0.5, 1).tolist()
points2 = np.arange(minToF+0.5, maxToF+0.5, 1).tolist()
ToF_list = list(zip(points1, points2)) 
print(f"Length of ToF_list: {len(ToF_list)}")

#ion_list = ['C1','C2','C3','C7(2+)','C4','IND(2+)','C5','C6','C7','IND(+)']

# Part (a): Getting data into a form in which it can be read directly into the functions above.

data_store = [] # to store PlottingArray for each dataframe, which can then be summed element-wise later
covar_contingent_store = [] # to store covariance maps for different FEL pulse energy intervals

number_of_bins = 2 # (n.b: actual number of bins is this value minus 1)

for df,file in zip(df_list,file_list):
    
    #print('Opening file: ' + file)
    #count = 1
    #df = df.to_numpy()
    
    #num_rows_total = np.shape(df)[0] # counts number of rows in numpy array of total data for iterating over later
    #print(f"Number of rows in total run number array: {num_rows_total}")
    #shot_array_total = np.unique(df[:,1]) # array of unique shot numbers in dataframe
    #print(f"Number of shots in total run number array: {len(shot_array_total)}")
    #FEL_bin_width = np.max(df[:,0])-np.min(df[:,0]) # this can be manually changed by the user, but make sure it's not too narrow to avoid a rubbish signal=to-noise ratio
    #FEL_energy_bins = np.arange(np.min(df[:,0]), np.max(df[:,0])+FEL_bin_width ,FEL_bin_width)
    #print(FEL_energy_bins)
    
    print(df)
    df = df.to_numpy()
    
    list_of_ion_frequencies_per_shot = [] # array of lists; each list will be of number of ion occurences (for a given ion) per shot
    ToF_list_for_plotting = [] # will be used to find midpoint of each tuple in ToF_list for plotting
        
    num_rows = np.shape(df)[0] # number of rows in new filtered df
    shot_array = np.unique(df[:,1]) # array of unique shot numbers (tId) in dataframe

    for i, (Ti, Tf) in enumerate(ToF_list): # considering each ion at a time

        ion_count_per_shot_list = np.zeros(len(shot_array)) # initialising array

        for row in range(num_rows): # iterating over rows in numpy data array

            if Ti <= df[row, 4] <= Tf: # if time-of-flight of a given row falls in the ion ToF range:

                shot_no = int(df[row,1]) # just extracts shot number
                index = int(np.where(shot_array == shot_no)[0])
                ion_count_per_shot_list[index] += 1

            else:
                pass

            
        ion_count_per_shot_list = list(ion_count_per_shot_list) # turning into list format
        list_of_ion_frequencies_per_shot.append(ion_count_per_shot_list) # list_of_ion_frequencies_per_shot becomes list of lists

        if (i+1)%10 == 0: # just progress print statements
            print(f"Formatting for ToF range no. {i+1} of {len(ToF_list)} complete.")
            
            
    print(f"*** Starting third cumulant calculations... ***")
    ### See comment in function for explanation of this output
    PlottingArray = GenerateThirdCumulantPlottingData(list_of_ion_frequencies_per_shot) # PlottingArray is a list
    #print(PlottingArray)
    ### Concatenate above function into form (1000,2) (for test data), where each row in the array has a cumulant value, 
    ### and an i value that corresponds to the i value in list_of_lists[i] it is produced from
    PlottingArray = np.concatenate(PlottingArray, axis =0)
    
    print(np.shape(PlottingArray))
    print(np.shape(PlottingArray[0]))
    print(PlottingArray[0])
    
    covar_contingent_store.append(PlottingArray) # covar_contingent_store become a list of lists for a given run number (refreshed after every run number)
    
    list_of_ion_frequencies_per_shot = [] # to clear for next iteration of FEL filters
    
    data_store.append([sum(x) for x in zip(*covar_contingent_store)]) # sum elements in list of lists elementwise
    
    print(f"Finished three-fold covariance calculations for file: {file}")
    
#print(np.shape(data_store))
#PlottingArray = np.sum(data_store, axis=0) # for run-aggregated PlottingArray - not needed as just one run
print(np.shape(PlottingArray))
print(np.shape(PlottingArray[0]))
print(PlottingArray[0])
    
print("Finished calculating all three-fold covariances.")

In [ ]:
# 6. Sorting out data into a way which can be plotted

import matplotlib

# indene 2023
ToF_list_reference = [(1630,1634),(1665,1670),(1685,1697),(1700,1703),(1708,1718),(1720,1722),(1725,1735),(1740,1750),(1755,1765),(1785,1795)]
ion_list = ['C1','C2','C3','C7(2+)','C4','IND(2+)','C5','C6','C7','IND(+)']

list_of_ToF_maps = [] # for all time intervals

for i in range(len(ToF_list)):
    filteredPlottingArray = [t for t in PlottingArray if int(t[1]) == i] # filter by i value
    list_of_cumulants = np.array([j[0] for j in filteredPlottingArray]) # extract only first value (cumulant) in filtered list of tuples
    matrix = list_of_cumulants.reshape((len(ToF_list),len(ToF_list))) # reshapes to plottable matrix
    list_of_ToF_maps.append(matrix)
    
    if (i+1)%10 == 0: # just progress print statements
        print(f"Formatting covariance map for ToF range no. {i+1} of {len(ToF_list)} complete.")
    

In [ ]:
# 7. Plotting!

from mpl_toolkits.axes_grid1 import make_axes_locatable

tof_appended = []
for df,file in zip(df_list,file_list):
    print('Opening file: '+str(file[0]))
    #df = df[(df['ToF']>1500) & (df['ToF']<1900)]
    tof = df.groupby(['ToF']).size().reset_index(name='intensity')
    tof_times_list = tof['ToF'].to_list()
    tof_appended.append(tof)

list_of_tof_intensities = []

for tof_frame in zip(tof_appended):
    tof_frame = tof_frame[0]
    tof_intensity = tof_frame['intensity'].to_list()
    list_of_tof_intensities.append(tof_intensity)

total_summed_intensities = [sum(i) for i in zip(*list_of_tof_intensities)]

fig, ax = plt.subplots(figsize=(15, 6))
plt.plot(tof_times_list,total_summed_intensities)

ax.set_xlim(1550,1850)
ax.grid(which='major')
ax.grid(which='minor')
ax.minorticks_on()
plt.xlabel('Time-of-flight / arb. units', fontsize=16)
plt.ylabel('Intensity / arb. units', fontsize=16)
plt.title('Total ToF spectrum')
#fig.savefig('triphenylene_total_tof.png')

tof_times_list = [x-minToF for x in tof_times_list]

for j, (Ti,Tf) in enumerate(ToF_list_reference):
    toflist = points1
    lower_index = toflist.index(Ti-0.5)
    upper_index = toflist.index(Tf-0.5)
    #print(lower_index, upper_index)
    plotting_list = list_of_ToF_maps[lower_index:upper_index]
    covar = sum(plotting_list)
    
    fig = plt.figure(figsize=(12, 12))
    gs = fig.add_gridspec(2, 2,  width_ratios=(3, 1), height_ratios=(1, 3), left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0, hspace=0)
    ax = fig.add_subplot(gs[1, 0])

    ax_x = fig.add_subplot(gs[0, 0], sharex=ax) # TOF plot (top row)
    ax_y = fig.add_subplot(gs[1, 1], sharey=ax) # TOF plot (column)

    ax_x.plot(tof_times_list,total_summed_intensities, color = 'purple')
    ax_y.plot(total_summed_intensities, tof_times_list, color = 'purple')

    #ax.set_xlim(0,300)
    ax_x.grid(which='major')
    ax_x.grid(which='minor', linewidth=0.3)
    ax_x.minorticks_on()
    ax_y.grid(which='major')
    ax_y.grid(which='minor', linewidth=0.3)
    ax_y.minorticks_on()
    ax_x.tick_params(axis="x",direction="out", pad=5)
    ax_y.tick_params(axis="y",direction="out", pad=5)
    ax.set_xlabel('Time-of-flight / a.u. ', fontsize=14,labelpad=15)
    ax.set_ylabel('Time-of-flight / a.u. ', fontsize=14,labelpad=15)
    ax_x.set_ylabel('Intensity / a.u. ', fontsize=14,labelpad=15)
    ax_y.set_xlabel('Intensity / a.u. ', fontsize=14,labelpad=15)      
    plt.setp(ax_x.get_xticklabels(), visible=False)
    plt.setp(ax_y.get_yticklabels(), visible=False)

    ax.grid(which='major')
    ax.grid(which='minor', linewidth=0.3)
    ax.minorticks_on()

    picture = ax.imshow(covar, interpolation='nearest', cmap='inferno', vmin=0, vmax=np.max(covar)/20) # edit vmax as necessary

    # colorbar formatting
    fig.subplots_adjust(right=0.8)
    #cbar_ax = fig.add_axes([1, 0.15, 0.05, 0.7])
    
    divider = make_axes_locatable(ax_y)
    cax = divider.append_axes("right", size="10%", pad=0.10)

    fig.colorbar(picture, cax=cax, pad = 0.1)
    cax.set_ylabel('Covariance / a.u.', fontsize = 14, rotation = 270, labelpad=25)
    
    ax_x.text(0.50, 1.10, f"3D TOF-TOF covariance plot (reference ion: {ion_list[j]})", horizontalalignment='center', verticalalignment='center', transform=ax_x.transAxes, fontsize = 14, weight='bold')

    shw = ax.imshow(covar, cmap='inferno', origin='lower',vmin=0, vmax=np.max(covar)/10)
    
    directory=os.chdir(r"/home/merrickj/Documents/indene_ion_figs_and_cov_plots")
    fig.savefig(f"TOF_3D_covariance_indene_ref_{j}",bbox_inches="tight")

    #directory=os.chdir(r"/home/merrickj/Documents/fluorene_ion_figs_and_cov_plots")
    #fig.savefig(f"TOF_3D_covariance_fluorene_ref_{j}",bbox_inches="tight")
    

In [ ]:
# Step 8: Test to see if event-rate (ions per shot) is Poisson-distributed to confirm reliability of using third 
#         cumulance. Makes sense to plot run-separated event-rate distributions for each shot since third cumulant code is
#         run on each run number in turn and not on a concatenated dataset.
    
for df,file in zip(df_list,file_list):
    
    fig, ax = plt.subplots(figsize=(15, 6))

    plt.xlabel('Ions per shot', fontsize=16)
    plt.tick_params(axis='x', labelsize=12)
    plt.ylabel('Frequency', fontsize=16)
    plt.tick_params(axis='y', labelsize=12)
    ax.grid(which='major')
    ax.grid(which='minor')
    ax.minorticks_on()
    
    plt.title(f"Run-separated event-rate distribution for file: {file}")
    event_rate = df.groupby(['tId']).size().reset_index(name='frequency')
    ions_in_shot = np.array(event_rate['frequency'])
    ax.hist(ions_in_shot, bins = 80)
    plt.show()
    
    mu_event = int(np.mean(ions_in_shot))
    print(f"Mean calculated for file {file}: {mu_event}")
    std_event= int(np.std(ions_in_shot))
    print(f"Standard deviation calculated for file {file}: {std_event}")
    
    fig.savefig(f"TOF_3D_test_if_poisson_indene_ref_{j}",bbox_inches="tight")
    #fig.savefig(f"TOF_3D_test_if_poisson_CPP_ref_{j}",bbox_inches="tight")

In [ ]:
# Step 5: Running the code! This is adapted with the possibility of contingent covariance, but this was found to severely reduce
#         the signal-to-noise ratio. By setting number of bins to 2, covariance is calculated without contingency. ***WIP

# N.B.: Contingent covariance NOT implemented in this code.

minToF = 1550
maxToF = 1800
points1 = np.arange(minToF-0.5, maxToF-0.5, 1).tolist()
points2 = np.arange(minToF+0.5, maxToF+0.5, 1).tolist()
ToF_list = list(zip(points1, points2)) 
print(f"Length of ToF_list: {len(ToF_list)}")

ion_list = ['C1','C2','C3','C7(2+)','C4','IND(2+)','C5','C6','C7','IND(+)']

# Part (a): Getting data into a form in which it can be read directly into the functions above.

covar_contingent_store = [] # to store covariance maps for different FEL pulse energy intervals

number_of_bins = 5 # for contingent covariance (in reality, number of bins is one minus from this value) (JHM)

for df in zip(df_list): # looping over each run number to sort energy bin stuff out
        
    df = df[0].sort_values(by=['Energy']) # load dataframe in ascending energy value
    df_length = df.shape[0] # number of rows in total dataframe

    min_energy = df['Energy'].min()
    max_energy = df['Energy'].max()

    FEL_energy_bins = []

    index_range = np.arange(0, df_length, df_length/number_of_bins)
    index_range_int = []

    for a in range(len(index_range)): # need to make all values integers
        index_range_int.append(int(index_range[a]))

    print(f"Integer index range: {index_range_int}")

    for index in index_range_int:
        indexed_energy = df['Energy'].iloc[index]
        FEL_energy_bins.append(indexed_energy)

    print(f"FEL energy bin values: {FEL_energy_bins}")

for df,file in zip(df_list,file_list):
    
    #print('Opening file: ' + file)
    #count = 1
    #df = df.to_numpy()
    
    #num_rows_total = np.shape(df)[0] # counts number of rows in numpy array of total data for iterating over later
    #print(f"Number of rows in total run number array: {num_rows_total}")
    #shot_array_total = np.unique(df[:,1]) # array of unique shot numbers in dataframe
    #print(f"Number of shots in total run number array: {len(shot_array_total)}")
    #FEL_bin_width = np.max(df[:,0])-np.min(df[:,0]) # this can be manually changed by the user, but make sure it's not too narrow to avoid a rubbish signal=to-noise ratio
    #FEL_energy_bins = np.arange(np.min(df[:,0]), np.max(df[:,0])+FEL_bin_width ,FEL_bin_width)
    #print(FEL_energy_bins)
    
    print(df)
    
    for j in range(len(FEL_energy_bins)- 1): # iterating over each energy bin filtered data
        
        print(f"*** Contingent covariance bin {j+1} of {len(FEL_energy_bins)}: ***")
        
        FEL_lower = FEL_energy_bins[j]
        FEL_higher = FEL_energy_bins[j+1]
        
        FEL_filtered_df = df[(df['Energy'] >= FEL_lower) & (df['Energy'] <= FEL_higher)]
        
        FEL_filtered_df = FEL_filtered_df.to_numpy()
    
        list_of_ion_frequencies_per_shot = [] # array of lists; each list will be of number of ion occurences (for a given ion) per shot
        ToF_list_for_plotting = [] # will be used to find midpoint of each tuple in ToF_list for plotting

        num_rows = np.shape(FEL_filtered_df)[0] # number of rows in new filtered df
        shot_array = np.unique(FEL_filtered_df[:,1]) # array of unique shot numbers (tId) in dataframe

        for i, (Ti, Tf) in enumerate(ToF_list): # considering each ion at a time

            ion_count_per_shot_list = np.zeros(len(shot_array)) # initialising array

            for row in range(num_rows): # iterating over rows in numpy data array

                if Ti <= FEL_filtered_df[row, 4] <= Tf: # if time-of-flight of a given row falls in the ion ToF range:

                    shot_no = int(FEL_filtered_df[row,1]) # just extracts shot number
                    index = int(np.where(shot_array == shot_no)[0])
                    ion_count_per_shot_list[index] += 1

                else:
                    pass


            ion_count_per_shot_list = list(ion_count_per_shot_list) # turning into list format
            list_of_ion_frequencies_per_shot.append(ion_count_per_shot_list) # list_of_ion_frequencies_per_shot becomes list of lists

            if (i+1)%10 == 0: # just progress print statements
                print(f"Formatting for ToF range no. {i+1} of {len(ToF_list)} complete.")


        print(f"*** Starting third cumulant calculations... ***")
        ### See comment in function for explanation of this output
        PlottingArray = GenerateThirdCumulantPlottingData(list_of_ion_frequencies_per_shot) # PlottingArray is a list
        #print(PlottingArray)
        ### Concatenate above function into form (1000,2) (for test data), where each row in the array has a cumulant value, 
        ### and an i value that corresponds to the i value in list_of_lists[i] it is produced from
        PlottingArray = np.concatenate(PlottingArray, axis =0)

        print(np.shape(PlottingArray))
        print(np.shape(PlottingArray[0]))
        print(PlottingArray[0])

        covar_contingent_store.append(PlottingArray) # covar_contingent_store become a list of lists for a given run number (refreshed after every run number)

        list_of_ion_frequencies_per_shot = [] # to clear for next iteration of FEL filters
    
    print(f"Finished three-fold covariance calculations for file: {file}")
    
#print(np.shape(data_store))
#PlottingArray = np.sum(data_store, axis=0) # for run-aggregated PlottingArray - not needed as just one run
print(np.shape(PlottingArray))
print(np.shape(PlottingArray[0]))
print(PlottingArray[0])
    
print("Finished calculating all three-fold covariances! :)")

In [ ]:
# 6. Sorting out data into a way which can be plotted.

import matplotlib

# indene 2023
ToF_list_reference = [(1630,1634),(1665,1670),(1685,1697),(1700,1703),(1708,1718),(1720,1722),(1725,1735),(1740,1750),(1755,1765),(1785,1795)]
ion_list = ['C1','C2','C3','C7(2+)','C4','IND(2+)','C5','C6','C7','IND(+)']

# fluorene 2023
#ToF_list_reference = [(1831,1840),(1820,1830),(1810,1818),(1798,1802),(1780,1792),(1770,1778),(1755,1765),(1750,1752),(1742,1749),(1735,1740),(1725,1733),(1720,1722),(1707,1715),(1699,1704),(1685,1698),(1662,1672),(1630,1633)]
#ion_list = ['FLU(+)','C12(+)','C11(+)','C10(+)','C9(+)','C8(+)','C7(+)','FLU(++)','C6(+)','C11(++)','C5(+)','FLU(+++)/C9(++)','C4(+)','C3(+)','C7(++)/FLU(++++)','C2(+)','C1(+)']

list_of_ToF_maps = [] # for all time intervals

for i in range(len(ToF_list)): # for each time interval for ALL time bins
    
    matrix_store = [] # for storing maps for a given time bin from different energy bins (from contingent)
    
    for PlottingArray in covar_contingent_store:
        
        filteredPlottingArray = [t for t in PlottingArray if int(t[1]) == i] # filter PlottingArray by i value
        list_of_cumulants = np.array([j[0] for j in filteredPlottingArray]) # extract only first value (cumulant) in filtered list of tuples
        matrix = list_of_cumulants.reshape((len(ToF_list),len(ToF_list))) # reshapes to plottable matrix
        matrix_store.append(matrix)
        
    matrix = sum(matrix_store)
    list_of_ToF_maps.append(matrix)
    
    if (i+1)%10 == 0: # just progress print statements
        print(f"Formatting covariance map for ToF range no. {i+1} of {len(ToF_list)} complete.")